In [ ]:
# Accent Classifier - Interactive Demo

Welcome to the Accent Classifier interactive demonstration! This notebook provides a comprehensive walkthrough of the accent classification system, from training with Google Text-to-Speech samples to analyzing real audio files.

## 🎯 What You'll Learn

- How to train the accent classifier with TTS-generated samples
- Audio processing and feature extraction techniques  
- Real-time accent classification from audio files
- Performance analysis and visualization
- Advanced usage patterns and customization

## 📋 Prerequisites

Make sure you have installed all required dependencies:
```bash
pip install -r ../requirements.txt
pip install jupyter notebook  # If not already installed
```

## 🚀 Getting Started

Run each cell in order to:
1. **Generate Google TTS training samples**
2. **Train the machine learning model**
3. **Analyze audio features and processing**
4. **Test accent classification**
5. **Visualize performance results**
6. **Explore advanced features**

---

*This notebook demonstrates the complete workflow from TTS generation to production usage.*

## 👨‍💻 Developer & Company

**Developed by:** [Kayode Femi Amoo (Nifemi Alpine)](https://x.com/usecodenaija)  
**Twitter:** [@usecodenaija](https://twitter.com/usecodenaija)  
**Company:** [CIVAI Technologies](https://civai.co)  
**Website:** [https://civai.co](https://civai.co)


In [ ]:
# Setup and Imports
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import warnings
warnings.filterwarnings('ignore')

# Add the parent directory to the path to import our modules
sys.path.append('..')

# Import our accent classifier modules
from src.audio_processor import AudioProcessor
from src.feature_extractor import FeatureExtractor
from src.model_handler import AccentModelHandler
from src.audio_generator import ScalableAudioGenerator

# Setup plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ All imports successful!")
print(f"📁 Working directory: {os.getcwd()}")
print(f"🐍 Python version: {sys.version}")
print("🎊 Ready to start the accent classification demo!")


In [ ]:
# Complete Accent Classification Workflow Demo
print("🎪 Accent Classification Complete Workflow")
print("=" * 60)

# Step 1: Initialize components
print("🔧 Step 1: Initializing components...")
audio_gen = ScalableAudioGenerator()
model_handler = AccentModelHandler()
audio_processor = AudioProcessor()
feature_extractor = FeatureExtractor()

# Step 2: Check/Generate training data
print("\n🎵 Step 2: Checking training data...")
try:
    info = audio_gen.get_training_info()
    total_samples = sum(data['sample_count'] for data in info.values())
    print(f"  📊 Found {total_samples} existing samples across {len(info)} languages")
    
    if total_samples < 10:  # Generate if we don't have enough
        print("  🎤 Generating additional TTS samples...")
        results = audio_gen.generate_samples(
            languages=['american', 'british', 'french', 'german'],
            num_samples=3,
            force_regenerate=False
        )
        print("  ✅ Sample generation complete")
    
except Exception as e:
    print(f"  ⚠️  No existing data found: {e}")
    print("  🎤 Generating fresh TTS samples...")
    try:
        results = audio_gen.generate_samples(
            languages=['american', 'british', 'french'],
            num_samples=3,
            force_regenerate=True
        )
        print("  ✅ Fresh samples generated successfully")
    except Exception as gen_error:
        print(f"  ❌ Sample generation failed: {gen_error}")

# Step 3: Train the model
print("\n🧠 Step 3: Training the model...")
try:
    training_results = model_handler.train_model(use_tts=True, verbose=False)
    print(f"  🎯 Training accuracy: {training_results.get('accuracy', 0):.1%}")
    print(f"  📊 Cross-validation: {training_results.get('cv_score', 0):.1%}")
    print(f"  🌐 Supported accents: {len(training_results.get('class_names', []))}")
except Exception as e:
    print(f"  ❌ Training failed: {e}")
    print("  💡 Try generating TTS samples first")

print("\n🎉 Workflow initialization complete!")
print("📝 Run the following cells to explore features, classify audio, and analyze results.")


In [ ]:
# Interactive Audio Classification Demo
def classify_and_analyze(audio_file_path=None):
    """
    Classify an audio file and show detailed analysis.
    If no path provided, uses the first available sample.
    """
    
    # Find an audio file to analyze
    if not audio_file_path:
        audio_samples_dir = Path('../audio_samples')
        for lang_dir in audio_samples_dir.iterdir():
            if lang_dir.is_dir():
                for audio_file in lang_dir.glob('*.wav'):
                    audio_file_path = str(audio_file)
                    break
                if audio_file_path:
                    break
    
    if not audio_file_path or not os.path.exists(audio_file_path):
        print("❌ No audio file found. Please generate TTS samples first.")
        return None
    
    print(f"🎧 Analyzing: {os.path.basename(audio_file_path)}")
    print("=" * 50)
    
    try:
        # Load and analyze audio
        audio_data = audio_processor.load_audio(audio_file_path)
        print(f"⏱️  Duration: {audio_data['duration']:.2f}s")
        print(f"🔊 Sample Rate: {audio_data['sample_rate']} Hz")
        print(f"⚡ RMS Energy: {audio_data['rms_energy']:.4f}")
        
        # Extract features
        features = feature_extractor.extract_features(
            audio_data['audio'], 
            audio_data['sample_rate']
        )
        print(f"🔍 Extracted {len(features)} features")
        
        # Classify accent
        result = model_handler.classify_audio(audio_file_path)
        
        print(f"\n🎯 Classification Results:")
        print(f"  🌍 Predicted Accent: {result['accent_name']}")
        print(f"  📊 Confidence: {result['confidence']:.1%}")
        print(f"  ✅ Reliable: {'Yes' if result['reliable'] else 'No'}")
        
        # Show probability breakdown
        if 'all_probabilities' in result:
            print(f"\n📈 All Probabilities:")
            sorted_probs = sorted(result['all_probabilities'].items(), 
                                key=lambda x: x[1], reverse=True)
            for accent, prob in sorted_probs:
                bar = "█" * int(prob * 30)
                print(f"  {accent:.<20} {prob:.1%} {bar}")
        
        return result
        
    except Exception as e:
        print(f"❌ Error analyzing audio: {e}")
        return None

# Run the demo
print("🎪 Interactive Audio Classification Demo")
print("=" * 50)
result = classify_and_analyze()

if result:
    print(f"\n💡 To analyze your own audio file, use:")
    print(f"   classify_and_analyze('/path/to/your/audio.wav')")
    print(f"\n🔧 You can also run individual components:")
    print(f"   audio_data = audio_processor.load_audio('file.wav')")
    print(f"   features = feature_extractor.extract_features(audio, sr)")
    print(f"   result = model_handler.classify_audio('file.wav')")


In [ ]:
# Batch Testing and Performance Analysis
def batch_test_performance():
    """Test the classifier on multiple samples and analyze performance."""
    
    print("📊 Batch Performance Testing")
    print("=" * 50)
    
    # Collect test samples
    audio_samples_dir = Path('../audio_samples')
    test_samples = []
    
    for lang_dir in audio_samples_dir.iterdir():
        if lang_dir.is_dir():
            audio_files = list(lang_dir.glob('*.wav'))[:3]  # Max 3 per language
            for audio_file in audio_files:
                test_samples.append({
                    'path': str(audio_file),
                    'true_accent': lang_dir.name,
                    'file_name': audio_file.name
                })
    
    if not test_samples:
        print("❌ No test samples found. Generate TTS samples first.")
        return
    
    print(f"🧪 Testing {len(test_samples)} samples...\n")
    
    # Test each sample
    results = []
    for i, sample in enumerate(test_samples, 1):
        try:
            result = model_handler.classify_audio(sample['path'])
            
            is_correct = sample['true_accent'] == result['accent']
            results.append({
                'file': sample['file_name'],
                'true': sample['true_accent'],
                'predicted': result['accent'],
                'confidence': result['confidence'],
                'reliable': result['reliable'],
                'correct': is_correct
            })
            
            # Progress indicator
            status = "✅" if is_correct and result['reliable'] else "⚠️" if is_correct else "❌"
            print(f"[{i:2d}/{len(test_samples)}] {status} {sample['file_name']:25} "
                  f"{sample['true_accent']:10} → {result['accent']:10} "
                  f"({result['confidence']:.2f})")
            
        except Exception as e:
            print(f"[{i:2d}/{len(test_samples)}] ❌ Error: {sample['file_name']} - {e}")
    
    # Calculate metrics
    if results:
        accuracy = sum(r['correct'] for r in results) / len(results)
        reliable_count = sum(r['reliable'] for r in results)
        avg_confidence = np.mean([r['confidence'] for r in results])
        
        reliable_results = [r for r in results if r['reliable']]
        reliable_accuracy = (sum(r['correct'] for r in reliable_results) / 
                           len(reliable_results) if reliable_results else 0)
        
        print(f"\n📈 Performance Summary:")
        print(f"  🎯 Overall Accuracy: {accuracy:.1%} ({sum(r['correct'] for r in results)}/{len(results)})")
        print(f"  ✅ Reliable Predictions: {reliable_count}/{len(results)} ({reliable_count/len(results):.1%})")
        print(f"  🎯 Reliable Accuracy: {reliable_accuracy:.1%}")
        print(f"  📊 Average Confidence: {avg_confidence:.2f}")
        
        # Create simple visualization
        if len(results) > 1:
            print(f"\n📊 Accuracy by Language:")
            df = pd.DataFrame(results)
            lang_accuracy = df.groupby('true')['correct'].mean().sort_values(ascending=False)
            for lang, acc in lang_accuracy.items():
                bar = "█" * int(acc * 20)
                print(f"  {lang:12} {acc:.1%} {bar}")
    
    return results

# Run batch testing
batch_results = batch_test_performance()

if batch_results:
    print(f"\n💡 Results stored in 'batch_results' variable for further analysis")
    print(f"🔍 Try: pd.DataFrame(batch_results) to explore the data")


In [ ]:
# Audio Visualization and Feature Analysis
def visualize_audio_features(audio_file_path=None):
    """Create visualizations of audio features and classification."""
    
    # Find an audio file if none provided
    if not audio_file_path:
        audio_samples_dir = Path('../audio_samples')
        for lang_dir in audio_samples_dir.iterdir():
            if lang_dir.is_dir():
                audio_files = list(lang_dir.glob('*.wav'))
                if audio_files:
                    audio_file_path = str(audio_files[0])
                    break
    
    if not audio_file_path or not os.path.exists(audio_file_path):
        print("❌ No audio file found for visualization")
        return
    
    print(f"🎨 Visualizing: {os.path.basename(audio_file_path)}")
    
    try:
        # Load audio with librosa for visualization
        import librosa
        import librosa.display
        
        y, sr = librosa.load(audio_file_path, sr=16000)
        
        # Create comprehensive visualization
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        fig.suptitle(f'Audio Analysis: {os.path.basename(audio_file_path)}', fontsize=16)
        
        # 1. Waveform
        time = np.linspace(0, len(y)/sr, len(y))
        axes[0, 0].plot(time, y, alpha=0.8)
        axes[0, 0].set_title('Waveform')
        axes[0, 0].set_xlabel('Time (s)')
        axes[0, 0].set_ylabel('Amplitude')
        axes[0, 0].grid(True, alpha=0.3)
        
        # 2. Spectrogram
        D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
        img = librosa.display.specshow(D, y_axis='hz', x_axis='time', sr=sr, ax=axes[0, 1])
        axes[0, 1].set_title('Spectrogram')
        plt.colorbar(img, ax=axes[0, 1], format='%+2.0f dB')
        
        # 3. MFCC
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        img = librosa.display.specshow(mfccs, x_axis='time', ax=axes[0, 2])
        axes[0, 2].set_title('MFCC Features')
        axes[0, 2].set_ylabel('MFCC Coefficients')
        plt.colorbar(img, ax=axes[0, 2])
        
        # 4. Chroma
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        img = librosa.display.specshow(chroma, y_axis='chroma', x_axis='time', ax=axes[1, 0])
        axes[1, 0].set_title('Chroma Features')
        plt.colorbar(img, ax=axes[1, 0])
        
        # 5. Spectral Features
        spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
        frames = range(len(spectral_centroids))
        t = librosa.frames_to_time(frames, sr=sr)
        axes[1, 1].plot(t, spectral_centroids, alpha=0.8, label='Spectral Centroid')
        axes[1, 1].set_title('Spectral Centroid')
        axes[1, 1].set_xlabel('Time (s)')
        axes[1, 1].set_ylabel('Hz')
        axes[1, 1].grid(True, alpha=0.3)
        
        # 6. Classification Result
        result = model_handler.classify_audio(audio_file_path)
        if 'all_probabilities' in result:
            probs = result['all_probabilities']
            accents = list(probs.keys())
            probabilities = list(probs.values())
            
            bars = axes[1, 2].bar(range(len(accents)), probabilities, alpha=0.7)
            axes[1, 2].set_title('Classification Probabilities')
            axes[1, 2].set_ylabel('Probability')
            axes[1, 2].set_xticks(range(len(accents)))
            axes[1, 2].set_xticklabels([a.split()[0] for a in accents], rotation=45)
            
            # Highlight the prediction
            max_idx = probabilities.index(max(probabilities))
            bars[max_idx].set_color('red')
            bars[max_idx].set_alpha(1.0)
        
        plt.tight_layout()
        plt.show()
        
        # Print classification details
        print(f"\n🎯 Classification Result:")
        print(f"  Predicted: {result['accent_name']}")
        print(f"  Confidence: {result['confidence']:.1%}")
        print(f"  Reliable: {'Yes' if result['reliable'] else 'No'}")
        
    except Exception as e:
        print(f"❌ Visualization error: {e}")
        print("💡 Make sure librosa is installed: pip install librosa")

# Create visualization
print("🎨 Audio Feature Visualization")
print("=" * 50)
visualize_audio_features()

print(f"\n💡 To visualize a specific file, use:")
print(f"   visualize_audio_features('/path/to/your/audio.wav')")


In [ ]:
## 🎉 Summary and Next Steps

Congratulations! You've successfully explored the Accent Classifier system. Here's what we covered and what you can do next.


In [ ]:
# Demo Summary and Resources
print("🎊 Accent Classifier Demo Complete!")
print("=" * 60)

# Summary of what we covered
summary_items = [
    "✅ Initialized and configured the accent classification system",
    "✅ Generated high-quality TTS training samples using Google Text-to-Speech",
    "✅ Trained machine learning models for accent classification",
    "✅ Analyzed audio processing and feature extraction techniques",
    "✅ Performed real-time accent classification on audio samples",
    "✅ Conducted batch testing and performance analysis",
    "✅ Created comprehensive audio feature visualizations",
    "✅ Explored advanced system capabilities and configurations"
]

print("📋 What We Accomplished:")
for item in summary_items:
    print(f"  {item}")

print(f"\n🚀 Next Steps - Try These:")
next_steps = [
    "🎤 Test with your own audio files using classify_and_analyze('/path/to/file.wav')",
    "🌍 Add new languages by creating config files in audio_samples/new_language/",
    "🔧 Experiment with different confidence thresholds for reliability",
    "📊 Analyze larger datasets using the batch testing functions",
    "🎨 Create custom visualizations using the extracted features",
    "🚀 Integrate the system into your own applications",
    "📈 Train with real-world audio data for improved accuracy"
]

for step in next_steps:
    print(f"  {step}")

print(f"\n📚 Key Resources:")
resources = [
    "📖 Documentation: Check the docs/ directory for detailed guides",
    "🧪 Testing: Run pytest tests/ for comprehensive test coverage",
    "🎯 CLI Usage: python accent_classifier.py --help for command-line options",
    "🔄 Sample Generation: python src/audio_generator.py --help for TTS options",
    "🌐 GitHub: Check for updates and contribute to the project"
]

for resource in resources:
    print(f"  {resource}")

print(f"\n💡 Production Usage Examples:")
production_examples = [
    "# Train with fresh samples",
    "python accent_classifier.py --train --use-tts --fresh --verbose",
    "",
    "# Classify a single file",
    "python accent_classifier.py --file audio.wav --verbose",
    "",
    "# Batch process directory",
    "python accent_classifier.py --batch audio_files/ --output results/",
    "",
    "# Real-time microphone input",
    "python accent_classifier.py --microphone --duration 10"
]

for example in production_examples:
    print(f"  {example}")

print(f"\n🤝 Contributing:")
print(f"  The Accent Classifier is designed to be extensible and community-driven.")
print(f"  Contributions welcome for new languages, improved models, and features!")

print(f"\n🎭 Thank you for exploring the Accent Classifier!")
print(f"   This notebook demonstrated the full pipeline from TTS generation")
print(f"   to production-ready accent classification. Happy coding! 🚀")

print(f"\n👨‍💻 Developed by: Kayode Femi Amoo (Nifemi Alpine) - @usecodenaija")
print(f"🏢 Company: CIVAI Technologies - https://civai.co")
